# Setups

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pprint import pprint
from IPython.display import display, Markdown

In [ ]:
import os
import sys

if cwd_is_root := ('examples' in os.listdir()):
    sys.path.append('examples')

In [ ]:
from pathlib import Path
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path('examples' if cwd_is_root else '.') / '.env')

# Imports

In [ ]:
from openssa import Agent, HTP, AutoHTPlanner, OodaReasoner, FileResource
from FinanceBench.data import cache_dir_path

# Problems & Resources

In [ ]:
PROBLEM = 'Does AMD have a healthy liquidity profile based on FY22 Quick Ratio?'

In [ ]:
RESOURCE_PATH = cache_dir_path(doc_name='AMD_2022_10K')
assert RESOURCE_PATH.is_dir()

resource = FileResource(RESOURCE_PATH)
display(Markdown(resource.overview))

# OODA Reasoning: 1 single pass

In [ ]:
ooda_reasoning_agent = Agent(planner=None,
                             reasoner=OodaReasoner(),
                             resources={resource})
pprint(ooda_reasoning_agent)

In [ ]:
display(Markdown(ooda_reasoning_agent.solve(PROBLEM)))

# Agent with Hierarchical Task Planning (HTP) & OODA Reasoning

In [ ]:
agent = Agent(planner=AutoHTPlanner(max_depth=2, max_subtasks_per_decomp=3),
              reasoner=OodaReasoner(),
              resources={resource})
pprint(agent)

## Problem-Solving with Automated Dynamic Planning (default)

In [ ]:
display(Markdown(agent.solve(PROBLEM)))

## Problem-Solving with Automated Static Planning

(this tends to produce the worst result)

In [ ]:
display(Markdown(agent.solve(PROBLEM, dynamic=False)))

## Problem-Solving with Expert-Specified Static Planning

In [ ]:
expert_plan = HTP.from_dict(
    {
        'task': PROBLEM,
        'sub-plans': [
            {
                'task': 'retrieve data points needed for Quick Ratio',
                'sub-plans': [
                    {
                        'task': 'retrieve Cash & Cash Equivalents'
                    },
                    {
                        'task': 'retrieve Accounts Receivable'
                    },
                    {
                        'task': 'retrieve Short-Term Liabilities'
                    },
                    {
                        'task': 'retrieve Accounts Payable'
                    },
                ]
            },
            {
                'task': 'calculate Quick Ratio'
            },
            {
                'task': 'see whether Quick Ratio is healthy, i.e. greater than 1'
            },
        ]
    }
)
pprint(expert_plan)

In [ ]:
display(Markdown(agent.solve(PROBLEM, plan=expert_plan, dynamic=False)))

## Problem-Solving with Expert-Guided Dynamic Planning

(this tends to product the best result)